In [44]:
import pandas as pd
import numpy as np
import xarray as xr
import os

#  Load  and prepare HW and NO_HW df

##  Load HW data

In [45]:
# Load the parquet data from /Trex/case_results/i.e215.I2000Clm50SpGs.hw_production.02/research_results/parquet
#data_dir = '/Users/yguo/DataSpellProjects/hw/data/parquet'
data_dir = '/Trex/case_results/i.e215.I2000Clm50SpGs.hw_production.02/research_results/parquet'
file_name_hw = 'HW_1985_1994.parquet'
#join data_dir and file_name
hw_path = os.path.join(data_dir, file_name_hw)
df_hw = pd.read_parquet(hw_path)
print(df_hw.info())
df_hw

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 20107224 entries, (-40.994766, 288.75, Timestamp('1985-12-01 00:00:00')) to (62.67016, 113.75, Timestamp('1994-07-15 23:00:00'))
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   UHI     float32
 1   UBWI    float32
 2   HW      float64
dtypes: float32(2), float64(1)
memory usage: 442.4 MB
None


UHI      UBWI   HW
lat        lon    time                                        
-40.994766 288.75 1985-12-01 00:00:00  1.284210 -0.116805  1.0
                  1985-12-01 01:00:00  1.196442  0.104045  1.0
                  1985-12-01 02:00:00  1.178253  0.174866  1.0
                  1985-12-01 03:00:00  1.142365  0.199996  1.0
                  1985-12-01 04:00:00  1.093079  0.209863  1.0
...                                         ...       ...  ...
 62.670158 113.75 1994-07-15 19:00:00  2.029236  1.313322  1.0
                  1994-07-15 20:00:00  1.745575  1.034362  1.0
                  1994-07-15 21:00:00  1.385864  0.682290  1.0
                  1994-07-15 22:00:00  0.643921  0.205750  1.0
                  1994-07-15 23:00:00  0.090027 -0.027581  1.0

[20107224 rows x 3 columns]

##  add hour, month and year to the df_hw


In [46]:
# Ensure 'time' is of datetime type
df_hw.index = df_hw.index.set_levels([df_hw.index.levels[0], df_hw.index.levels[1], pd.to_datetime(df_hw.index.levels[2])])

# Extract hour, month and year from 'time'
df_hw['hour'] = df_hw.index.get_level_values('time').hour
df_hw['month'] = df_hw.index.get_level_values('time').month
df_hw['year'] = df_hw.index.get_level_values('time').year
df_hw

UHI      UBWI   HW  hour  month  \
lat        lon    time                                                        
-40.994766 288.75 1985-12-01 00:00:00  1.284210 -0.116805  1.0     0     12   
                  1985-12-01 01:00:00  1.196442  0.104045  1.0     1     12   
                  1985-12-01 02:00:00  1.178253  0.174866  1.0     2     12   
                  1985-12-01 03:00:00  1.142365  0.199996  1.0     3     12   
                  1985-12-01 04:00:00  1.093079  0.209863  1.0     4     12   
...                                         ...       ...  ...   ...    ...   
 62.670158 113.75 1994-07-15 19:00:00  2.029236  1.313322  1.0    19      7   
                  1994-07-15 20:00:00  1.745575  1.034362  1.0    20      7   
                  1994-07-15 21:00:00  1.385864  0.682290  1.0    21      7   
                  1994-07-15 22:00:00  0.643921  0.205750  1.0    22      7   
                  1994-07-15 23:00:00  0.090027 -0.027581  1.0    23      7   

                                       year  
lat        lon    time                       
-40.994766 288.75 1985-12-01 00:00:00  1985  
                  1985-12-01 01:00:00  1985  
                  1985-12-01 02:00:00  1985  
                  1985-12-01 03:00:00  1985  
                  1985-12-01 04:00:00  1985  
...                                     ...  
 62.670158 113.75 1994-07-15 19:00:00  1994  
                  1994-07-15 20:00:00  1994  
                  1994-07-15 21:00:00  1994  
                  1994-07-15 22:00:00  1994  
                  1994-07-15 23:00:00  1994  

[20107224 rows x 6 columns]

In [47]:
# # Group by 'lat', 'lon', 'year', 'month', and 'hour', then calculate the mean of 'UHI' and 'UBWI'
# df_hw_avg = df_hw.groupby(['lat', 'lon', 'year', 'month', 'hour']).mean()
# df_hw_avg

##  Load NO_HW data

In [48]:

file_name_no_hw = 'NO_HW_1985_1994.parquet'
#join data_dir and file_name
no_hw_path = os.path.join(data_dir, file_name_no_hw)
df_no_hw = pd.read_parquet(no_hw_path)
print(df_no_hw.info())
df_no_hw

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 73117416 entries, (-55.13089, 291.25, Timestamp('1985-01-01 00:00:00')) to (70.20943, 31.25, Timestamp('1994-08-31 23:00:00'))
Data columns (total 2 columns):
 #   Column  Dtype  
---  ------  -----  
 0   UHI     float32
 1   UBWI    float32
dtypes: float32(2)
memory usage: 1.1 GB
None


UHI      UBWI
lat        lon    time                                   
-55.130890 291.25 1985-01-01 00:00:00 -0.465027 -0.488769
                  1985-01-01 01:00:00  0.007538  0.027193
                  1985-01-01 02:00:00  0.213013  0.172712
                  1985-01-01 03:00:00  0.209503  0.155361
                  1985-01-01 04:00:00  0.164490  0.133307
...                                         ...       ...
 70.209427 31.25  1994-08-31 19:00:00  0.879242  0.335660
                  1994-08-31 20:00:00  1.104370  0.431603
                  1994-08-31 21:00:00  1.207367  0.512312
                  1994-08-31 22:00:00  1.184082  0.508102
                  1994-08-31 23:00:00  1.111176  0.470229

[73117416 rows x 2 columns]

In [49]:
#todo: add UHI and NO_HW UHI to make sure they are the same as the oringal netcdf data


##  Validate there is not overlap between the HW and NO_HW data

In [50]:
#the key for both df_hw and df_no_hw are lat, lon and time. please show python code that they don't overlap on those keys
# Convert the MultiIndex of both DataFrames to sets
# keys_hw = set(df_hw.index)
# keys_no_hw = set(df_no_hw.index)
# 
# # Check if the intersection of these sets is empty
# overlap = keys_hw & keys_no_hw
# 
# # If the intersection is empty, print that there is no overlap. Otherwise, print the overlapping keys.
# if not overlap:
#     print("There is no overlap between the keys of df_hw and df_no_hw.")
# else:
#     print("The following keys overlap between df_hw and df_no_hw:")
#     print(overlap)

In [51]:

# group df_no_hw by lat, lon, year and hour of the day avaerage UHI and UBWI
df_no_hw.index = df_no_hw.index.set_levels(
    [df_no_hw.index.levels[0], df_no_hw.index.levels[1], pd.to_datetime(df_no_hw.index.levels[2])])
df_no_hw['hour'] = df_no_hw.index.get_level_values('time').hour
df_no_hw['year'] = df_no_hw.index.get_level_values('time').year
df_no_hw_avg = df_no_hw.groupby(['lat', 'lon', 'year', 'hour']).mean()
df_no_hw_avg

UHI      UBWI
lat        lon    year hour                    
-55.130890 291.25 1985 0     0.305542  0.122376
                       1     0.366234  0.171864
                       2     0.374915  0.186239
                       3     0.351239  0.173043
                       4     0.331068  0.164803
...                               ...       ...
 70.209427 31.25  1994 19    0.677809  0.283742
                       20    0.815796  0.349291
                       21    0.902967  0.396645
                       22    0.955209  0.448562
                       23    0.972885  0.492617

[1002504 rows x 2 columns]

#  Calculate the difference between UHI in df_hw and df_no_hw

##  UHI HW - NO_HW ( HW hour data - NO_HW yearl average data for the hour) 
the df_no_hw_avg is the average value for a given hour of the day throughout the year.
In the 2018 Zhao paper they seem to just do average the whole 30 years. 
I want to substract the average UHI on the given hour for a given year from the hourly UHI data I have in df_hw, matching the year and hour between the two dataframes.


##  Step 1: Reset the index of df_hw and df_no_hw_avg (be careful on the increased memory usage)

In [63]:
df_hw.info()
df_hw_reset = df_hw.reset_index()
df_hw_reset.info()
df_no_hw_avg.info()
df_no_hw_avg_reset = df_no_hw_avg.reset_index()
df_no_hw_avg_reset.info()


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 20107224 entries, (-40.994766, 288.75, Timestamp('1985-12-01 00:00:00')) to (62.67016, 113.75, Timestamp('1994-07-15 23:00:00'))
Data columns (total 6 columns):
 #   Column  Dtype  
---  ------  -----  
 0   UHI     float32
 1   UBWI    float32
 2   HW      float64
 3   hour    int32  
 4   month   int32  
 5   year    int32  
dtypes: float32(2), float64(1), int32(3)
memory usage: 672.5 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20107224 entries, 0 to 20107223
Data columns (total 9 columns):
 #   Column  Dtype         
---  ------  -----         
 0   lat     float32       
 1   lon     float32       
 2   time    datetime64[ns]
 3   UHI     float32       
 4   UBWI    float32       
 5   HW      float64       
 6   hour    int32         
 7   month   int32         
 8   year    int32         
dtypes: datetime64[ns](1), float32(4), float64(1), int32(3)
memory usage: 843.7 MB
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 10


##   Step 2: Merge df_hw with df_no_hw_avg_reset

In [53]:
df_hw

UHI      UBWI   HW  hour  month  \
lat        lon    time                                                        
-40.994766 288.75 1985-12-01 00:00:00  1.284210 -0.116805  1.0     0     12   
                  1985-12-01 01:00:00  1.196442  0.104045  1.0     1     12   
                  1985-12-01 02:00:00  1.178253  0.174866  1.0     2     12   
                  1985-12-01 03:00:00  1.142365  0.199996  1.0     3     12   
                  1985-12-01 04:00:00  1.093079  0.209863  1.0     4     12   
...                                         ...       ...  ...   ...    ...   
 62.670158 113.75 1994-07-15 19:00:00  2.029236  1.313322  1.0    19      7   
                  1994-07-15 20:00:00  1.745575  1.034362  1.0    20      7   
                  1994-07-15 21:00:00  1.385864  0.682290  1.0    21      7   
                  1994-07-15 22:00:00  0.643921  0.205750  1.0    22      7   
                  1994-07-15 23:00:00  0.090027 -0.027581  1.0    23      7   

                                       year  
lat        lon    time                       
-40.994766 288.75 1985-12-01 00:00:00  1985  
                  1985-12-01 01:00:00  1985  
                  1985-12-01 02:00:00  1985  
                  1985-12-01 03:00:00  1985  
                  1985-12-01 04:00:00  1985  
...                                     ...  
 62.670158 113.75 1994-07-15 19:00:00  1994  
                  1994-07-15 20:00:00  1994  
                  1994-07-15 21:00:00  1994  
                  1994-07-15 22:00:00  1994  
                  1994-07-15 23:00:00  1994  

[20107224 rows x 6 columns]

In [57]:

merged_df = pd.merge(df_hw, df_no_hw_avg_reset[['lat', 'lon', 'year', 'hour', 'UHI']],
                     on=['lat', 'lon', 'year', 'hour'],
                     suffixes=('', '_avg'))

MergeError: Can only pass argument "on" OR "left_index" and "right_index", not a combination of both.

In [56]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18740520 entries, 0 to 18740519
Data columns (total 9 columns):
 #   Column   Dtype  
---  ------   -----  
 0   lat      float32
 1   lon      float32
 2   UHI      float32
 3   UBWI     float32
 4   HW       float64
 5   hour     int32  
 6   month    int32  
 7   year     int32  
 8   UHI_avg  float32
dtypes: float32(5), float64(1), int32(3)
memory usage: 714.9 MB



##  Step 3: Subtract the average UHI from the hourly UHI and store in a new column

In [38]:

merged_df['UHI_diff'] = merged_df['UHI'] - merged_df['UHI_avg']

# Now, merged_df contains your original data along with the subtracted UHI values in 'UHI_diff'
merged_df  # To check the first few rows of the merged DataFrame

,lat,lon,UHI,UBWI,HW,hour,month,year,UHI_avg,UHI_diff
0,-40.994766,288.75,1.284210,-0.116805,1.0,0,12,1985,0.949050,0.335160
1,-40.994766,288.75,1.196442,0.104045,1.0,1,12,1985,0.949663,0.246778
2,-40.994766,288.75,1.178253,0.174866,1.0,2,12,1985,0.920274,0.257979
3,-40.994766,288.75,1.142365,0.199996,1.0,3,12,1985,0.864492,0.277873
4,-40.994766,288.75,1.093079,0.209863,1.0,4,12,1985,0.805318,0.287760
...,...,...,...,...,...,...,...,...,...,...
18740515,62.670158,113.75,0.643921,0.205750,1.0,22,7,1994,1.093816,-0.449895
18740516,62.670158,113.75,0.894958,-2.112110,1.0,23,7,1994,-0.198605,1.093564
18740517,62.670158,113.75,0.939423,-0.827865,1.0,23,7,1994,-0.198605,1.138028
18740518,62.670158,113.75,1.404633,-0.092875,1.0,23,7,1994,-0.198605,1.603238


## Step 4: Validate the results by checking the UHI values for a specific location and time

In [39]:

merged_df.iloc[0].lat
a_row = merged_df.iloc[59]
df_no_hw_avg_reset.loc[(df_no_hw_avg_reset['lat'] == a_row.lat) & (df_no_hw_avg_reset['lon'] == a_row.lon) & (
                df_no_hw_avg_reset['year'] == a_row.year) & (
                df_no_hw_avg_reset['hour'] == a_row.hour)]


,lat,lon,year,hour,UHI,UBWI
8883,-39.109947,291.25,1985,3,0.758839,-0.135349


In [ ]:
import pandas as pd
import numpy as np

def convert_index_time_to_local(df):
    """
    Converts the UTC timestamp in the DataFrame's MultiIndex to local time based on longitude.
    
    Parameters:
    df (pd.DataFrame): DataFrame with a MultiIndex (latitude, longitude, UTC timestamp)
    
    Returns:
    pd.DataFrame: DataFrame with the timestamp in the MultiIndex adjusted to local time
    """
    # Function to calculate timezone offset from longitude
    def calculate_timezone_offset(longitude):
        return np.floor(longitude / 15.0).astype(int)  # Approximate, not accounting for DST or specific timezone rules

    # Extract MultiIndex levels
    latitudes, longitudes, timestamps = zip(*df.index)

    # Convert to arrays for vectorized operations
    longitudes = np.array(longitudes)
    timestamps = pd.to_datetime(list(timestamps))

    # Calculate timezone offsets
    offsets = calculate_timezone_offset(longitudes)

    # Adjust timestamps by the offsets
    adjusted_timestamps = [timestamp + pd.Timedelta(hours=offset) for timestamp, offset in zip(timestamps, offsets)]

    # Create a new MultiIndex with the adjusted timestamps
    new_index = pd.MultiIndex.from_arrays([latitudes, longitudes, adjusted_timestamps], names=['Latitude', 'Longitude', 'LocalTime'])

    # Assign the new MultiIndex to the DataFrame and return it
    df.index = new_index
    return df

# Usage example:
# new_df = convert_index_time_to_local(df)
# This will return a DataFrame with the index timestamps adjusted to local time based on longitude
# Note: ONLY CALL THIS FUNCTION ON THE DATAFRAME WITH THE MULTIINDEX (latitude, longitude, UTC timestamp), ALSO ONLY CALL AT THE FINAL STEP(AFTER ALL AGGREATION DONE!!!



In [ ]:
df_hw = convert_index_time_to_local(df_hw)
df_hw

In [ ]:
df_count = df_hw.groupby(['lat', 'lon']).size()
df_count

In [ ]:
import os
os.environ["PROJ_LIB"] = "/home/jguo/anaconda3/envs/I2000/share/proj"

import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

df_hw = pd.read_parquet('/Trex/case_results/i.e215.I2000Clm50SpGs.hw_production.02/research_results/parquet/HW_1985_1994.parquet')
df_count = df_hw.groupby(['lat', 'lon']).size()
df_count_sorted = df_count.sort_values(ascending=False).head(50)
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE)

# Add gridlines
gl = ax.gridlines(draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False

lats, lons = zip(*df_count_sorted.index)
ax.scatter(lons, lats, color='red', transform=ccrs.PlateCarree())

plt.show()

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(df_count_sorted, geometry=gpd.points_from_xy(lons, lats))

# Load Natural Earth dataset with coastlines
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Plot the world (coastlines)
fig, ax = plt.subplots(figsize=(10, 5))
world.boundary.plot(ax=ax, color='black')

# Plot the GeoDataFrame
gdf.plot(marker='o', color='red', markersize=5, ax=ax)
plt.show()

In [ ]:
df_count_sorted.info()

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np

# Create a plot with coastlines
fig = plt.figure(figsize=(10, 5))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

# Extract latitudes, longitudes, and data values from the MultiIndex Series
latitudes = df_count_sorted.index.get_level_values(0)
longitudes = df_count_sorted.index.get_level_values(1) % 360  # Adjust longitudes if they are in a 0-360 format
data_values = df_count_sorted.values

# Plot the data
scatter = plt.scatter(longitudes, latitudes, c=data_values, cmap='viridis', transform=ccrs.PlateCarree())

# Add gridlines and labels
gl = ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False

# Add a colorbar
cbar = plt.colorbar(scatter, orientation='vertical', pad=0.02, aspect=50)
cbar.set_label('Data Value')

plt.show()




In [ ]:
# Convert the Series with a MultiIndex to an xarray DataArray
da = xr.DataArray(df_count_sorted, dims=['lat_lon'], name='data')
da['lat'] = ('lat_lon', df_count_sorted.index.get_level_values('lat'))
da['lon'] = ('lat_lon', df_count_sorted.index.get_level_values('lon'))
# Define the new grid (adjust the resolution as needed)
new_lat = np.linspace(da['lat'].min(), da['lat'].max(), num=100)
new_lon = np.linspace(da['lon'].min(), da['lon'].max(), num=100)

# Interpolate the DataArray onto the new grid
da_interp = da.interp(lat=new_lat, lon=new_lon)


In [ ]:
da.HW_Count.plot()


In [ ]:

df_no_hw = pd.read_parquet('/Trex/case_results/i.e215.I2000Clm50SpGs.hw_production.02/research_results/parquet/NO_HW_1985_1994.parquet')

print(df_no_hw.info())  


In [ ]:
df_hw.index.levels[2]

In [ ]:
# Ensure 'time' is of datetime type
df_hw.index = df_hw.index.set_levels([df_hw.index.levels[0], df_hw.index.levels[1], pd.to_datetime(df_hw.index.levels[2])])

# Extract hour, month and year from 'time'
df_hw['hour'] = df_hw.index.get_level_values('time').hour
df_hw['month'] = df_hw.index.get_level_values('time').month
df_hw['year'] = df_hw.index.get_level_values('time').year

# Group by 'lat', 'lon', 'year', 'month', and 'hour', then calculate the mean of 'UHI' and 'UBWI'
df_avg = df_hw.groupby(['lat', 'lon', 'year', 'month', 'hour']).mean()
df_avg